In [1]:
from tensorflow.keras.models import load_model
import numpy as np

In [2]:
import cv2

In [3]:
model = load_model('final.h5')

In [4]:
from skimage.transform import resize

In [5]:
def detect(frame):
    try:
        img = resize(frame,(64, 64))
        img = np.expand_dims(img, axis = 0)
        if(np.max(img) > 1):
            img = img/255.0
        prediction = model.predict(img)
        return model.predict_classes(img)
    except AttributeError:
        print("Shape not found")
        

# {'Normal': 0, 'Robbery': 1, 'Vandalism': 2}

In [6]:
import smtplib
from email.message import EmailMessage
def email_alert(body):
    msg = EmailMessage()
    text = "Dear User,\nPlease check your surveillance video.\nIt seems like "+body.upper()+" is happening!"
    msg.set_content(text)
    msg['subject'] = 'Alert!'
    msg['to'] = 'To_ACCOUNT'
    
    user = "ACCOUNT_NAME"
    password = "PASSWORD"
    msg['from'] = user
    
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(user,password)
    server.send_message(msg)
    
    server.quit()
    

In [7]:
import os
from twilio.rest import Client

def msg_alert(body):
    account_sid = os.environ["YOUR_TWILIO_ACCOUNT_SID"]
    auth_token = os.environ["YOUR_TWILIO_AUTH_TOKEN"]
    client = Client(account_sid,auth_token)
    text = "ALERT\nDear user,please check your surveillance video, It seems like "+body+" is happening!"

    client.messages.create(to=os.environ["YOUR_PHONE_NUMBER"],
                      from_="YOUR_TWILIO_NUMBER",
                      body=text)

In [8]:
import numpy as np
import cv2
v = 1   #for vandalism
r = 1   #for robbery

cap = cv2.VideoCapture(0)

frame_width = int( cap.get(cv2.CAP_PROP_FRAME_WIDTH))

frame_height =int( cap.get( cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc('X','V','I','D')

out = cv2.VideoWriter("output.mp4", fourcc, 5.0, (1280,720))

while(True):
    ret, frame = cap.read()

    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(frame, (1280,720))
    out.write(image)
    d = detect(frame)
    
    
    if d == 2:
        cv2.putText(frame,"Status:{}".format('Vandalism'),(50,50), cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
       
        if v==1:
            msg_alert('Vandalism')
            email_alert("Vandalism")
            v=0
            
    elif d == 1:
        cv2.putText(frame,"Status:{}".format('Robbery'),(50,50), cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        
        if r==1:
            msg_alert('Robbery')
            email_alert("Robbery")
            r=0
        
    
    # Display the resulting frame
    cv2.imshow('OUTPUT',frame)
    
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
